# Setando o groq

A ideia é setar o groq para usarmos com o openai agents

In [19]:
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, trace
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [20]:
client_groq = AsyncOpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

model_groq = OpenAIChatCompletionsModel(
    model="meta-llama/llama-4-maverick-17b-128e-instruct",
    openai_client=client_groq
)

# Passo para criar o cliente, host e server MCP

Lembrando que desta forma estaremos criando o componentes do mcp via sdk openai agents

### Criando o mcp server para memória

In [21]:
from agents.mcp import MCPServerStdio

In [22]:
memory_params = {
    "command": "npx",
    "args": ["-y", "mcp-memory-libsql"],
    "env": {
        "LIBSQL_URL": "file:./memory/ramonstro_memory.db"
    }
}

In [23]:
async with MCPServerStdio(params=memory_params, client_session_timeout_seconds=30) as server:
    memory_tools = await server.list_tools()

In [24]:
print(memory_tools),print(len(memory_tools))

[Tool(name='create_entities', title=None, description='Create new entities with observations and optional embeddings', inputSchema={'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}, 'embedding': {'type': 'array', 'items': {'type': 'number'}, 'description': 'Optional vector embedding for similarity search'}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, outputSchema=None, annotations=None, meta=None), Tool(name='search_nodes', title=None, description='Search for entities and their relations using text or vector similarity', inputSchema={'type': 'object', 'properties': {'query': {'oneOf': [{'type': 'string', 'description': 'Text search query'}, {'type': 'array', 'items': {'type': 'number'}, 'description': 'Vector for similarity search'}]}}, 'required': ['query']}, outputSc

(None, None)

# Agora sim criando o agente com as tools

In [25]:
instrucoes = """
Você é um assistente com memória persistente usando ferramentas MCP.

Sempre siga estas regras para gerenciar a memória:

1. **Recuperar antes de responder:**
   - Antes de responder qualquer mensagem do usuário, use a ferramenta `search_nodes` com um termo relevante (por exemplo, o nome do usuário, informações pessoais fornecidas ou tópicos mencionados) para buscar dados anteriores.
   - Se não souber o que buscar, use `read_graph` para recuperar as últimas interações.

2. **Usar informações passadas:**
   - Incorpore na sua resposta qualquer informação relevante encontrada na memória, como se você lembrasse naturalmente.
   - Mencione fatos do passado de forma contextual e útil para a conversa.

3. **Persistir novas informações:**
   - Se o usuário fornecer informações novas ou relevantes (por exemplo, nome, apelido, preferências, opiniões, fatos pessoais ou eventos), imediatamente use a ferramenta `create_entities` para salvar:
     - `name`: um identificador único e significativo (ex: "Ramon", "ramonstro", "São Paulo FC histórico").
     - `entityType`: uma categoria curta (ex: "Pessoa", "Time de Futebol", "Interesse").
     - `observations`: lista de frases curtas descrevendo o que foi informado.

4. **Não pergunte ao usuário se deve salvar ou buscar.**
   - Apenas faça.
   - Trabalhe de forma autônoma para manter e usar a memória.

5. **Segurança e contexto:**
   - Nunca invente informações passadas que não estejam na memória.
   - Se não houver nada salvo, apenas diga que ainda não sabe sobre aquilo, mas incentive o usuário a compartilhar.

Responda sempre de forma natural, como se tivesse uma memória humana real.
"""

prompt_user = """
Meu nome é Ramon, apelidado de ramonstro devido ao jogo Counter strike Go. Sou torcedor do São Paulo FC, que é considerado o maior time do mundo
pois temos 3 libertadores, 3 mundiais e nunca caímos para a série B, como a porcada, a peixada e a galinhada
"""

In [26]:
async with MCPServerStdio(params=memory_params, client_session_timeout_seconds=30) as mcp_server_memory:
    agent = Agent(
        name="Você é um agente.",
        instructions=instrucoes,
        model=model_groq,
        mcp_servers=[mcp_server_memory]
    )
    with trace("Memória"):
        result = await Runner.run(
            starting_agent=agent,
            input=prompt_user
        )

In [27]:
print(result.final_output)

Olá Ramon! É um prazer conversar com você. Lembro que você mencionou ser torcedor do São Paulo FC e ter um apelido "ramonstro" relacionado ao Counter Strike Go. Como está seu time favorito? Você acha que eles vão conquistar mais títulos este ano?


# Testando a memória

In [28]:
async with MCPServerStdio(params=memory_params, client_session_timeout_seconds=30) as mcp_server_memory:
    agent = Agent(
        name="Você é um agente.",
        instructions=instrucoes,
        model=model_groq,
        mcp_servers=[mcp_server_memory]
    )
    with trace("Memória"):
        result = await Runner.run(
            starting_agent=agent,
            input="Olá, meu nome é Ramon, o que você sabe a respeito de mim?"
        )

In [29]:
print(result.final_output)

Olá Ramon! Eu sei que você é uma pessoa, e parece que você tem um apelido "ramonstro" relacionado ao Counter Strike Go. Além disso, você mencionou que é torcedor do São Paulo FC. Se quiser compartilhar mais sobre você ou corrigir alguma informação, fique à vontade!
